In [0]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import imageio
from sklearn import svm
# import sklearn.svm.libsvm as svm
# import sklearn.svm as svm

In [0]:
!git clone https://github.com/KavishBhatia/MachineLearning.git

fatal: destination path 'MachineLearning' already exists and is not an empty directory.


In [0]:
category = ["positives", "negatives"]
ml = "MachineLearning/"

def getPositiveImages():
  image_path = []
  for i in range(1,31):
    if i < 10:
      img_name = "/p0" + str(i)
    else:
      img_name = "/p" +str(i)    
    path = ml + category[0] + img_name + ".png"
    image_path.append(path)
  return image_path  

def getNegativeImages():
  image_path = []
  for i in range(1,31):
    if i < 10:
      img_name = "/n0" + str(i)
    else:
      img_name = "/n" +str(i)    
    path = ml + category[1] + img_name + ".png"
    image_path.append(path)
  return image_path

In [0]:
picturePathListPositive = getPositiveImages()
picturePathListNegative = getNegativeImages()

picturePathList = picturePathListPositive + picturePathListNegative
# picturePathList

In [0]:
red = 0
green = 1
blue = 2
m = 60 # total number of pos and neg images
n = 3 #no of features

def getPixelValues(imagepath):
  im = imageio.imread(imagepath)
  return im

def computeMeanForAllColours(imagepath):
  pixValues = getPixelValues(imagepath)
  meanPixValues = np.mean(pixValues)
  return meanPixValues

def computeMeanSingleColour(colour, imagepath):
  pixValues = getPixelValues(imagepath)
  meanPixValues = np.mean(pixValues[:,:,colour])
  return meanPixValues

def filterMaxMean(image):
  img = Image.open(image)
  members = [(0,0)] * 4
  filteredMeanMax = 255
  for i in range(1,23):
    for j in range(1,23):
        members[0] = img.getpixel((i-1,j-1))
        members[1] = img.getpixel((i-1,j))
        members[2] = img.getpixel((i-1,j+1))
        members[3] = img.getpixel((i,j-1))
        tempFilteredMeanMax = np.mean(members)
        if tempFilteredMeanMax < filteredMeanMax:
          filteredMeanMax = tempFilteredMeanMax
  return (filteredMeanMax - 255)*(-1)

def calcMeanEdges(imagepath):
    image = Image.open(imagepath)
    return np.mean(image.filter(ImageFilter.FIND_EDGES))
  
def getFeaturesOneImage(imagepath):
    X = []
    X.append(filterMaxMean(imagepath))
    X.append(computeMeanForAllColours(imagepath))
    X.append(computeMeanSingleColour(red, imagepath))
    X.append(computeMeanSingleColour(blue, imagepath))
    X.append(calcMeanEdges(imagepath))
    X = np.asarray(X).reshape(1,n)
    return X

In [0]:
def getFeaturesOneImage(imagepath):
    X = []
    X.append((filterMaxMean(imagepath)))
    X.append((computeMeanSingleColour(blue, imagepath)))
    X.append((calcMeanEdges(imagepath)))
    #X = np.asarray(X).reshape(1,n)
    return X
  
SVMDataArray = []

for i in picturePathList:
    SVMDataArray.append(getFeaturesOneImage(i))
    
labelList = np.ones(30)
labelListZeroes = np.zeros(30)
labelList = np.append(labelList, labelListZeroes)
labelList = labelList.tolist()

# SVMDataArray

In [0]:
# from sklearn import svm
SupportVM = svm.SVC(kernel='linear') # kernel names: ‘linear’, ‘poly’, ‘sigmoid’, ‘rbf’
SupportVM.fit(SVMDataArray, labelList)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
SupportVM.predict([[135.8, 158.1, 40.]])


array([1.])

In [0]:
from sklearn.model_selection import KFold



SVMDataArray = np.array(SVMDataArray)
labelList = np.array(labelList)
kf = KFold(n_splits=10)
kf.get_n_splits(SVMDataArray)

counterLin = 0
counterPoly = 0
counterSig = 0
counterRad = 0

for train_index, test_index in kf.split(SVMDataArray):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print(SVMDataArray[train_index])
    X_train, X_test = SVMDataArray[train_index], SVMDataArray[test_index]
    y_train, y_test = labelList[train_index], labelList[test_index]
    
    svmLinear = svm.SVC(kernel='linear', gamma='auto')
    svmPoly = svm.SVC(kernel='poly', gamma='auto')
    svmSigmoid = svm.SVC(kernel='sigmoid', gamma='auto')
    svmRadial = svm.SVC(kernel='rbf', gamma=.01)
  
    svmLinear.fit(X_train, y_train)
    svmPoly.fit(X_train, y_train)
    svmSigmoid.fit(X_train, y_train)
    svmRadial.fit(X_train, y_train)
    
    for i in range(6):
      predictionLin = svmLinear.predict([X_test[i]])
      predictionPoly = svmPoly.predict([X_test[i]])
      predictionSig = svmSigmoid.predict([X_test[i]])
      predictionRad = svmRadial.predict([X_test[i]])
      testLabel = y_test[i]
      
      if predictionLin == testLabel:
        counterLin += 1
      
      if predictionPoly == testLabel:
        counterPoly += 1
        
      if predictionSig != testLabel:
        counterSig += 1
        
      if predictionRad == testLabel:
        counterRad += 1

print('Results for Lin:', counterLin, 
      'Results for Poly:', counterPoly,
      'Results for Sig:', counterSig,
      'Results for Rad:', counterRad)

# print (kf)
# print(svmLinear.predict([[135.8, 158.1, 40.]]))
# print(svmPoly.predict([[135.8, 158.1, 40.]]))
# print(svmSigmoid.predict([[135.8, 158.1, 40.]]))
# print(svmRadial.predict([[135.8, 158.1, 40.]]))
  

Results for Lin: 60 Results for Poly: 59 Results for Sig: 60 Results for Rad: 60
